In [40]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.optim as optim
from sklearn.model_selection import train_test_split
import pandas as pd

In [41]:
'''
    CODE TO PREPARE DATA.
'''
train_path = '/Users/rjaditya/Documents/projects/Kaggle/digit/dataset/train.csv'
test_path = '/Users/rjaditya/Documents/projects/Kaggle/digit/dataset/test.csv'

dataset = pd.read_csv(train_path, header=0)
submission_dataset = pd.read_csv(test_path, header=0)

y = dataset['label']
X = dataset.drop(labels='label', axis=1)
x_train, x_test, y_train, y_test = train_test_split(
  X,y , random_state=104,test_size=0.25, shuffle=True)

x_test.astype(float)
x_train.astype(float)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
19584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35641,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1081,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28559,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31240,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39078,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
train_data = x_train
train_data['label'] = y_train

test_data = x_test
test_data['label'] = y_test

In [43]:
train_dataset = torch.tensor(train_data.values)
test_dataset = torch.tensor(test_data.values)

In [44]:
import numpy as np
trainloader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=4,
                                          shuffle=False, num_workers=2)
testloader = torch.utils.data.DataLoader(test_dataset,
                                         shuffle=False, num_workers=2)

In [45]:
model = nn.Sequential(
    nn.Linear(in_features=783, out_features=500),
    nn.ReLU(),
    nn.Linear(in_features=500, out_features=250),
    nn.ReLU(),
    nn.Linear(in_features=250, out_features=10),
    nn.Softmax()
)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)
loss = nn.BCELoss

In [46]:
torch.__version__

'2.2.0.post100'

In [48]:
training_loss = []
from tqdm import tqdm
for epoch in tqdm(range(200)):  # loop over the dataset multiple times
    running_loss = 0
    for i, data in enumerate(trainloader):
        x, y = data[0], data[1]
        optimizer.zero_grad()
        y_hat = torch.argmax(model(x))
        tloss = loss(y_hat, y)
        tloss.backward()
        optimizer.step()
        running_loss += tloss.item()
    training_loss.append(running_loss)
    

  0%|          | 0/200 [00:01<?, ?it/s]


RuntimeError: mat1 and mat2 must have the same dtype, but got Long and Float